<a href="https://colab.research.google.com/github/Shisiajoy/BreastCancerPrediction/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### LOAD THE MODEL AND QUANTIZE

In [ ]:
import numpy as np
import tensorflow as tf


In [1]:
# Load your trained model
model = tf.keras.models.load_model('/content/my_model.keras', compile=False)

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_model = converter.convert()

# Save the quantized model
with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_model)


Saved artifact at '/tmp/tmpfto68fds'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132631755458512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631755462208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631755472592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631755722064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631755729456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631755731744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631913409072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631913511248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631755999536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132631756002352: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [3]:
# Load the quantized model
interpreter = tf.lite.Interpreter(model_path='quantized_model.tflite')

# Allocate tensors for the model
interpreter.allocate_tensors()

# Get input and output details (if you want to see the structure)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Quantized model loaded and ready for inference!")


Quantized model loaded and ready for inference!


In [5]:
# Assuming  input image is a (150, 150, 1) grayscale image
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150), color_mode='grayscale')
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array.astype('float32') / 255.0  # Normalize to [0, 1]
    return img_array

# Path to your input image
image_path = '/content/10008_1591370361 (1).png'
input_data = preprocess_image(image_path)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the model
interpreter.invoke()

# Get the prediction result
output_data = interpreter.get_tensor(output_details[0]['index'])
#print("Prediction:", output_data)


# Assuming 'output_data' contains the prediction probability
prediction_probability = output_data[0][0]  # Extract the predicted probability

# Define threshold (e.g., 0.5)
threshold = 0.5

# Determine the class based on the threshold
if prediction_probability >= threshold:
    prediction_class = "Cancer"
else:
    prediction_class = "No Cancer"

# Display the result
print(f"Prediction: {prediction_class} with a probability of {prediction_probability:.2f}")



Prediction: Cancer with a probability of 0.98
